# Pawpularity with EfficientNetB2 FineTuning

## Table of Contents
- Summary
- Set up
- Import datasets
- Data Preprocessing
- Model Development
- Model Evaluation
- Submission


## Summary
In this Notebook, I will:
* Use EfficientNetB2 as Image Model to fit on Image Data.
* Use DNN as Tabular Model to fit on Tabular Data.
* Combine the total result of Image Model and Tabular Model.
* Use Data Augmentation and Regularization method to prevent overfiting.
* Use K-Fold training to improve final score.

## Set up

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from tensorflow import keras

## Import datasets

In [ ]:
train = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
sample_submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")

## Data Preprocessing

In [ ]:
train.head()

In [ ]:
train["file_path"] = train["Id"].apply(lambda identifier: "../input/petfinder-pawpularity-score/train/" + identifier + ".jpg")
test["file_path"] = test["Id"].apply(lambda identifier: "../input/petfinder-pawpularity-score/test/" + identifier + ".jpg")

In [ ]:
train.head()

### Distribution of Pawpularities

In [ ]:
train["Pawpularity"].hist()

## Model Development

In [ ]:
tabular_columns = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
image_size = 260
batch_size = 128
epochs = 10
AUTO = tf.data.experimental.AUTOTUNE
output_dataset_path = "/kaggle/input/pawpularity-with-efficientnetb2-finetuning-output/"
modes = ["training", "inference"]
mode = modes[1]
is_trainging_mode = mode == modes[0]
folds = [0, 1, 2, 3, 4]

### Random Blockout Augmentation

In [ ]:
def random_erasing(img, sl=0.1, sh=0.2, rl=0.4, p=0.3):
    h = tf.shape(img)[0]
    w = tf.shape(img)[1]
    c = tf.shape(img)[2]
    origin_area = tf.cast(h*w, tf.float32)

    e_size_l = tf.cast(tf.round(tf.sqrt(origin_area * sl * rl)), tf.int32)
    e_size_h = tf.cast(tf.round(tf.sqrt(origin_area * sh / rl)), tf.int32)

    e_height_h = tf.minimum(e_size_h, h)
    e_width_h = tf.minimum(e_size_h, w)

    erase_height = tf.random.uniform(shape=[], minval=e_size_l, maxval=e_height_h, dtype=tf.int32)
    erase_width = tf.random.uniform(shape=[], minval=e_size_l, maxval=e_width_h, dtype=tf.int32)

    erase_area = tf.zeros(shape=[erase_height, erase_width, c])
    erase_area = tf.cast(erase_area, tf.uint8)

    pad_h = h - erase_height
    pad_top = tf.random.uniform(shape=[], minval=0, maxval=pad_h, dtype=tf.int32)
    pad_bottom = pad_h - pad_top

    pad_w = w - erase_width
    pad_left = tf.random.uniform(shape=[], minval=0, maxval=pad_w, dtype=tf.int32)
    pad_right = pad_w - pad_left

    erase_mask = tf.pad([erase_area], [[0,0],[pad_top, pad_bottom], [pad_left, pad_right], [0,0]], constant_values=1)
    erase_mask = tf.squeeze(erase_mask, axis=0)
    erased_img = tf.multiply(tf.cast(img,tf.float32), tf.cast(erase_mask, tf.float32))

    return tf.cond(tf.random.uniform([], 0, 1) > p, lambda: tf.cast(img, img.dtype), lambda:  tf.cast(erased_img, img.dtype))

In [ ]:
def data_augment(image):
    image = tf.image.random_flip_left_right(image)
    image = random_erasing(image)
    return image

### Preprocess funciton
I am doing data augmentation on training preprocess function, so that when make evaluation on validation dataset and prediction on test dataset, data augmentation won't be triggered.

In [ ]:
def preprocess_training(image_url, tabular):
    image_string = tf.io.read_file(image_url)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = data_augment(image)
    image = tf.image.central_crop(image, 1.0)
    image = tf.image.resize(image, (image_size, image_size))
    return (image, tabular[1:]), tf.cast(tabular[0], tf.float32)

def preprocess_validation(image_url, tabular):
    image_string = tf.io.read_file(image_url)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.central_crop(image, 1.0)
    image = tf.image.resize(image, (image_size, image_size))
    return (image, tabular[1:]), tf.cast(tabular[0], tf.float32)

## The Base Model (Efficient Net)

In [ ]:
def base_model():
    efficient_net = tf.keras.applications.EfficientNetB2(
        include_top = False, 
        input_shape = (image_size, image_size, 3)
    )    
    efficient_net.trainable = False
    return efficient_net

### The Tabular Prediction Model

I wrote a notebook to find optimal Tabualr Prediciton Model [here](https://www.kaggle.com/lonnieqin/pawpularity-model-with-dnn-on-meta-data?scriptVersionId=79014727).

In [ ]:
def get_tabular_model(inputs):
    width = 32
    depth = 3
    activation = "relu"
    kernel_regularizer = keras.regularizers.l2()
    x = keras.layers.Dense(
            width, 
            activation=activation,
            kernel_regularizer=kernel_regularizer
        )(inputs)
    for i in range(depth):
        if i == 0:
            x = inputs
        x = keras.layers.Dense(
            width, 
            activation=activation,
            kernel_regularizer=kernel_regularizer
        )(x)
        if (i + 1) % 3 == 0:
            x = keras.layers.Concatenate()([x, inputs])
    return x

In [ ]:
def get_model():
    image_inputs = tf.keras.Input((image_size, image_size , 3))
    tabular_inputs = tf.keras.Input(len(tabular_columns))
    efficient_net = base_model()
    image_x = efficient_net(image_inputs)
    image_x = tf.keras.layers.GlobalAveragePooling2D()(image_x)
    image_x = tf.keras.layers.Dropout(0.5)(image_x)
    tabular_x = get_tabular_model(tabular_inputs)
    x = tf.keras.layers.Concatenate(axis=1)([image_x, tabular_x])
    output = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(inputs=[image_inputs, tabular_inputs], outputs=[output])
    return model

### Model Training
I will use tensorflow Dataset here to preprocess and cache tensors, first epoch is very slow because it's preprocessing data; after that, it would be must faster.

In [ ]:
tf.keras.backend.clear_session()
models = []
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
for index, (train_indices, val_indices) in enumerate(kfold.split(train)):
    if index not in folds: 
        continue
    if not is_trainging_mode:
        break
    x_train = train.loc[train_indices, "file_path"]
    tabular_train = train.loc[train_indices, ["Pawpularity"] + tabular_columns]
    x_val= train.loc[val_indices, "file_path"]
    tabular_val = train.loc[val_indices, ["Pawpularity"] + tabular_columns]
    checkpoint_path = "model_%d.h5"%(index)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path, 
        save_best_only=True
    )
    early_stop = tf.keras.callbacks.EarlyStopping(
        min_delta=1e-4, 
        patience=10
    )
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        factor=0.3,
        patience=2, 
        min_lr=1e-5
    )
    callbacks = [early_stop, checkpoint, reduce_lr]
    
    optimizer = tf.keras.optimizers.Adam(3e-4)
    
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, tabular_train)).map(preprocess_training).shuffle(512).batch(batch_size).cache().prefetch(AUTO)
    val_ds = tf.data.Dataset.from_tensor_slices((x_val, tabular_val)).map(preprocess_validation).batch(batch_size).cache().prefetch(AUTO)
    model = keras.models.load_model(output_dataset_path +"model_%d.h5"%(index))
    efficientnetb2 = model.get_layer("efficientnetb2")
    efficientnetb2.trainable = True
    for i, layer in enumerate(efficientnetb2.layers):
        if i < len(efficientnetb2.layers) - 20:
            layer.trainable = False
        else:
            if type(layer) != keras.layers.BatchNormalization:
                layer.trainable = True
            else:
                layer.trainable = False
    rmse = tf.keras.metrics.RootMeanSquaredError(name="root_mean_squared_error")
    model.compile(loss="mse", optimizer=optimizer, metrics=["mae", "mape", rmse])
    history = model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=callbacks)
    model.load_weights(checkpoint_path)
    models.append(model)

## Submission

In [ ]:
def preprocess_test_data(image_url, tabular):
    print(image_url, tabular)
    image_string = tf.io.read_file(image_url)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.central_crop(image, 1.0)
    image = tf.image.resize(image, (image_size, image_size))
    return (image, tabular), 0

In [ ]:
test_ds = tf.data.Dataset.from_tensor_slices((test["file_path"], test[tabular_columns])).map(preprocess_test_data).batch(batch_size).cache().prefetch(AUTO)

In [ ]:
total_results = []
if is_trainging_mode:
    for model in models:
        total_results.append(model.predict(test_ds).reshape(-1))
else:
    for i in folds:
        model = keras.models.load_model(output_dataset_path +"model_%d.h5"%(i))
        total_results.append(model.predict(test_ds).reshape(-1))
results = np.mean(total_results, axis=0).reshape(-1)
sample_submission["Pawpularity"] = results
sample_submission.to_csv("submission.csv", index=False)